# Introduction
<font color = green> <strong>MUST READ:</strong></font> This Notebook tutorial will allow you to perform Asymmetric Authentication aka Node Authentication with TFLXTLS device. 

In this use case example, we will authenticate an Object/Node. It can be an accessory, peripheral, battery, or cartridge. Generally, an object that is removable and replaceable by the consumer. The purpose of authenticating an object is to ensure that it is genuine and it is authorized to connect to a product. Another purpose is to prevent cloning and counterfeiting. Asymmetric authentication uses asymmetric key algorithms (also known as public key cryptography) where each entity has a public and private key.

Node authentication occurs between two devices in a host-client configuration when the client’s identity must be verified before its connection to the host can be established.

### Prerequisites:
This step of the tutorial will attempt to load all the necessary modules and their dependencies on your machine. If the modules are already installed you can safely step over the next Tutorial step.

<font color = orange> <strong>Note</strong></font> 
1. Installation time for prerequisites depends upon system and network speed.
2. Installing prerequisites for the first time takes more time and watch the kernel status for progress. Following image helps to locate the Kernel status,
<center><img src="../../../assets/notebook/img/kerner_status.png" alt="**Check Kernel Status**" /></center>


3. Installing prerequisites gives the following error and it can be safely ignored. Functionality remains unaffected.
    - <font color = orange> azure-cli 2.0.76 has requirement colorama~=0.4.1, but you'll have colorama 0.3.9 which is incompatible.</font>
    - <font color = orange>  azure-cli 2.0.76 has requirement pytz==2019.1, but you'll have pytz 2019.3 which is incompatible. </font>

In [ ]:
import sys, os

home_path = os.path.dirname(os.path.dirname(os.path.dirname(os.path.realpath(os.getcwd()))))
module_path = os.path.join(home_path, 'assets', 'python')
if not module_path in sys.path:
    sys.path.append(module_path)

from requirements_helper import requirements_installer
obj = requirements_installer(os.path.join(home_path, 'assets', 'requirements.txt'))

### Setup Modules and Hardware
This step loads the required modules and helper functions to perform the resource generation sequence. It also 
1. Initializes the interface with TFLXTLS hardware and establishes commmunication with TFLXTLS.
2. Performs device initialization to verify the actual device attached

In [ ]:
import warnings
import ipywidgets as widgets

from ipywidgets import FileUpload
from base64 import b16encode
from cryptoauthlib import *
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.serialization import Encoding, PublicFormat
from cryptography.hazmat.primitives.asymmetric import ec, utils
from cryptography.utils import int_from_bytes

from certs_handler import *
from trustplatform import *
from asymm_auth_helper import *

layout=widgets.Layout(width='auto')
warnings.filterwarnings('ignore')
print('Importing modules - Successful')

#TFLXTLS device I2C address 0x6C; #TNGTLS device I2C address 0x6A; 
common_helper.connect_to_secure_element('ATECC608', ATCAKitType.ATCA_KIT_I2C_IFACE, 0x6C)
print("Device initialization - Successful")

### Verify Certificate Chain

This is split into multiple steps. The process of verification includes Reading (Client Build) both Signer and Device Certificates from Secure element, Signer and Device Certificates verification by host (Host Verify) and verify that the Device holds the associated Private key (Verify Device Private Key).

#### FIGURE: Verifying the Node

<left><img src="img/authenticating_the_node.png" alt="" /></center>

In [ ]:
def process_node_authentication():
    ####################################################################
    print('\nclient rebuild certificates...')
    try:
        device_cert_def, device_cert = rebuild_cert(root_cert_object, signer_cert_object, device_cert_object)
    except:
        return 'danger'
    
    ####################################################################
    print('\nhost verify cert chain...')
    assert verify_cert_chain(), 'Certificate chain is invalid!'

    ####################################################################
    print('host generate challenge...', end='')
    challenge = os.urandom(32)
    print('OK(Challenge: {})'.format(challenge.hex().upper()))

    ####################################################################
    print('client generate response...', end='')
    response = bytearray(64)
    assert atcacert_get_response(device_cert_def.private_key_slot, challenge, response) == CertStatus.ATCACERT_E_SUCCESS
    print('OK(Response: {})'.format(response.hex().upper()))

    ####################################################################
    print('host_verify_response:')
    r = int_from_bytes(response[0:32], byteorder='big', signed=False)
    s = int_from_bytes(response[32:64], byteorder='big', signed=False)
    sign = utils.encode_dss_signature(r, s)
    try:
        device_cert.public_key().verify(sign, challenge, ec.ECDSA(utils.Prehashed(hashes.SHA256())))
        print('\tNode has associated private key!\n')
    except:
        print('\tNode failed to prove its private key!')
        return 'danger'
    
    print('Node authentication completed successfully.')
    print('------------------------------------------------------')
    return 'success'

def node_authenticate_cb(b):
    node_authenticate.button_style = 'danger'
    node_authenticate.button_style = process_node_authentication()

root_cert_object = FileUpload(description='Select Root cert', accept='.crt', layout=widgets.Layout(width='auto'), multiple=False)
signer_cert_object = FileUpload(description='Select Signer cert', accept='.crt', layout=widgets.Layout(width='auto'), multiple=False)
device_cert_object = FileUpload(description='Select Device cert Template', accept='.crt', layout=widgets.Layout(width='auto'), multiple=False)
tooltip='Click to execute Node authentication steps'
node_authenticate = widgets.Button(description='Authenticate Node',tooltip=tooltip)
node_authenticate.on_click(node_authenticate_cb)

print('Select certificate templates for cert def files...')
display(widgets.VBox((root_cert_object, signer_cert_object, device_cert_object)))
display(node_authenticate)